In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pandas import DataFrame
from pycocotools.coco import COCO
import numpy as np

/opt/conda/envs/detection/lib/python3.7/site-packages/mmcv/__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
cfg = Config.fromfile('/opt/ml/baseline/mmdetection/configs/train/swin-l.py')

root='../../SGK_dataset_v2/fold0/'

epoch = '12'

# dataset config 수정
cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'
cfg.data.test.test_mode = True

cfg.data.samples_per_gpu = 4

cfg.seed=641178613
cfg.gpu_ids = [1]
cfg.work_dir = '/opt/ml/baseline/mmdetection/work_dirs/swin-l/'  # 파일 경로 수정 필요

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [3]:
# build dataset & dataloader
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=16,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:
# checkpoint path
checkpoint_path = os.path.join(cfg.work_dir, f'epoch_{epoch}.pth')

model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg')) # build detector
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load

model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/baseline/mmdetection/work_dirs/swin-l/epoch_12.pth


In [5]:
output = single_gpu_test(model, data_loader, show_score_thr=0.1) # output 계산

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 4871/4871, 4.6 task/s, elapsed: 1069s, ETA:     0s

In [6]:
output[0][0]

array([[1.6036797e+02, 4.2627948e+02, 2.0471629e+02, 4.5023004e+02,
        1.7300887e-01],
       [4.4963062e+02, 5.5010419e+02, 5.7374841e+02, 6.6223834e+02,
        9.9424720e-02],
       [4.7502179e+02, 5.8628314e+02, 5.7225604e+02, 6.5384186e+02,
        9.0446025e-02],
       [3.9113391e+02, 6.7854425e+02, 4.2229163e+02, 7.1943048e+02,
        8.6455919e-02],
       [5.3748083e+02, 6.3487097e+02, 5.7321814e+02, 6.5933936e+02,
        6.6822715e-02],
       [5.3414233e+02, 6.3808264e+02, 5.9015491e+02, 6.5370862e+02,
        6.1053660e-02],
       [5.3251691e+02, 6.4288354e+02, 5.8758636e+02, 6.5810608e+02,
        2.8797995e-02],
       [1.5830226e+02, 4.2822147e+02, 2.0679308e+02, 4.5027524e+02,
        2.0568091e-02]], dtype=float32)

In [7]:
# submission 양식에 맞게 output 후처리
#coco format : (xmin, ymin, w, h)
#pascal format : (label, score, xmin, ymin, xmax, ymax)
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

score_threshold = 0.1
class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            if o[4] > score_threshold:
                prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=None)
submission.head()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


,PredictionString,image_id
0,0 0.17300887 160.36797 426.27948 204.7163 450....,test/0000.jpg
1,0 0.66726476 634.41833 663.1121 679.97156 846....,test/0001.jpg
2,0 0.9591421 881.05444 454.00806 1011.28613 633...,test/0002.jpg
3,0 0.058986727 0.0 0.0 1024.0 375.10153 9 0.954...,test/0003.jpg
4,0 0.7923221 201.38095 271.82178 871.155 780.40...,test/0004.jpg
